In [1]:
import pandas as pd
import numpy as np

### Loading Data

In [2]:
train = pd.read_csv('../OriginalDataset/train.csv')
test =  pd.read_csv('../OriginalDataset/test.csv')
building = pd.read_csv('../OriginalDataset/building_metadata.csv')
weather_train = pd.read_csv('../OriginalDataset//weather_train.csv')
weather_test = pd.read_csv('../OriginalDataset//weather_test.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   meter          int64  
 2   timestamp      object 
 3   meter_reading  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 616.9+ MB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   row_id       int64 
 1   building_id  int64 
 2   meter        int64 
 3   timestamp    object
dtypes: int64(3), object(1)
memory usage: 1.2+ GB


In [5]:
building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   site_id      1449 non-null   int64  
 1   building_id  1449 non-null   int64  
 2   primary_use  1449 non-null   object 
 3   square_feet  1449 non-null   int64  
 4   year_built   675 non-null    float64
 5   floor_count  355 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 68.1+ KB


In [6]:
weather_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             139773 non-null  int64  
 1   timestamp           139773 non-null  object 
 2   air_temperature     139718 non-null  float64
 3   cloud_coverage      70600 non-null   float64
 4   dew_temperature     139660 non-null  float64
 5   precip_depth_1_hr   89484 non-null   float64
 6   sea_level_pressure  129155 non-null  float64
 7   wind_direction      133505 non-null  float64
 8   wind_speed          139469 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


In [7]:
weather_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277243 entries, 0 to 277242
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             277243 non-null  int64  
 1   timestamp           277243 non-null  object 
 2   air_temperature     277139 non-null  float64
 3   cloud_coverage      136795 non-null  float64
 4   dew_temperature     276916 non-null  float64
 5   precip_depth_1_hr   181655 non-null  float64
 6   sea_level_pressure  255978 non-null  float64
 7   wind_direction      264873 non-null  float64
 8   wind_speed          276783 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 19.0+ MB


### Filtering train and test set only for valid electrictiy readings (meter = 0)

In [8]:
train_electricity = train[(train['meter'] == 0) & (train['meter_reading']> 0)]
test_electricity = test[(test['meter'] == 0)]

In [9]:
train_electricity.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11530741 entries, 45 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   meter          int64  
 2   timestamp      object 
 3   meter_reading  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 439.9+ MB


### Merging everything into two datasets: train_electiricity and test_electricity

In [10]:
train_electricity_building = train_electricity.merge(building, on='building_id', how='left')
test_electricity_building = test_electricity.merge(building, on='building_id', how='left')

train_electricity_building.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11530741 entries, 0 to 11530740
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int64  
 1   meter          int64  
 2   timestamp      object 
 3   meter_reading  float64
 4   site_id        int64  
 5   primary_use    object 
 6   square_feet    int64  
 7   year_built     float64
 8   floor_count    float64
dtypes: float64(3), int64(4), object(2)
memory usage: 791.8+ MB


In [11]:
train_electricity_building_weather = train_electricity_building.merge(weather_train, on=['site_id', 'timestamp'], how='left')
test_electricity_building_weather = test_electricity_building.merge(weather_test, on=['site_id', 'timestamp'], how='left')

train_electricity_building_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11530741 entries, 0 to 11530740
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   building_id         int64  
 1   meter               int64  
 2   timestamp           object 
 3   meter_reading       float64
 4   site_id             int64  
 5   primary_use         object 
 6   square_feet         int64  
 7   year_built          float64
 8   floor_count         float64
 9   air_temperature     float64
 10  cloud_coverage      float64
 11  dew_temperature     float64
 12  precip_depth_1_hr   float64
 13  sea_level_pressure  float64
 14  wind_direction      float64
 15  wind_speed          float64
dtypes: float64(10), int64(4), object(2)
memory usage: 1.4+ GB


### Saving some memory


In [12]:
def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [13]:
reduce_memory_usage(train_electricity_building_weather)
reduce_memory_usage(test_electricity_building_weather)

Mem. usage decreased to 505.84 Mb (64.1% reduction)
Mem. usage decreased to 1086.01 Mb (64.1% reduction)


,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,2008.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,2004.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,1991.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,2002.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,1975.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24755755,41697595,1444,0,2018-05-09 07:00:00,15,Entertainment/public assembly,19619,1914.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24755756,41697596,1445,0,2018-05-09 07:00:00,15,Education,4298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24755757,41697597,1446,0,2018-05-09 07:00:00,15,Entertainment/public assembly,11265,1997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24755758,41697598,1447,0,2018-05-09 07:00:00,15,Lodging/residential,29775,2001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
train_electricity_building_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11530741 entries, 0 to 11530740
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   building_id         int16  
 1   meter               int8   
 2   timestamp           object 
 3   meter_reading       float32
 4   site_id             int8   
 5   primary_use         object 
 6   square_feet         int32  
 7   year_built          float16
 8   floor_count         float16
 9   air_temperature     float16
 10  cloud_coverage      float16
 11  dew_temperature     float16
 12  precip_depth_1_hr   float16
 13  sea_level_pressure  float16
 14  wind_direction      float16
 15  wind_speed          float16
dtypes: float16(9), float32(1), int16(1), int32(1), int8(2), object(2)
memory usage: 505.8+ MB


In [15]:
test_electricity_building_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24755760 entries, 0 to 24755759
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   row_id              int32  
 1   building_id         int16  
 2   meter               int8   
 3   timestamp           object 
 4   site_id             int8   
 5   primary_use         object 
 6   square_feet         int32  
 7   year_built          float16
 8   floor_count         float16
 9   air_temperature     float16
 10  cloud_coverage      float16
 11  dew_temperature     float16
 12  precip_depth_1_hr   float16
 13  sea_level_pressure  float16
 14  wind_direction      float16
 15  wind_speed          float16
dtypes: float16(9), int16(1), int32(2), int8(2), object(2)
memory usage: 1.1+ GB


### Saving Datasets

In [17]:
train_electricity_building_weather.to_csv('../FilteredDataset/train.csv')
test_electricity_building_weather.to_csv('../FilteredDataset/test.csv')